In [1]:
# 기본적인 부분
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc("font", family="Malgun Gothic")
plt.rcParams["axes.unicode_minus"]=False

# 데이터 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# 학습 알고리즘
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report
from scipy.special import expit, softmax

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

from sklearn.ensemble import RandomForestClassifier

# 서울시 상권별 서비스업 매출 예측

## 데이터 로드 및 결합  

### 서울시 상권 추정매출

In [2]:
seoul_sales=pd.read_csv("../data/서울시 상권분석서비스(추정매출-상권).csv", encoding="cp949")
seoul_sales.shape

(87179, 55)

In [3]:
seoul_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87179 entries, 0 to 87178
Data columns (total 55 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년분기_코드        87179 non-null  int64  
 1   상권_구분_코드         87179 non-null  object 
 2   상권_구분_코드_명       87179 non-null  object 
 3   상권_코드            87179 non-null  int64  
 4   상권_코드_명          87179 non-null  object 
 5   서비스_업종_코드        87179 non-null  object 
 6   서비스_업종_코드_명      87179 non-null  object 
 7   당월_매출_금액         87179 non-null  float64
 8   당월_매출_건수         87179 non-null  int64  
 9   주중_매출_금액         87179 non-null  float64
 10  주말_매출_금액         87179 non-null  float64
 11  월요일_매출_금액        87179 non-null  float64
 12  화요일_매출_금액        87179 non-null  float64
 13  수요일_매출_금액        87179 non-null  float64
 14  목요일_매출_금액        87179 non-null  float64
 15  금요일_매출_금액        87179 non-null  float64
 16  토요일_매출_금액        87179 non-null  float64
 17  일요일_매출_금액   

In [6]:
seoul_sales["서비스_업종_코드_명"].unique()

array(['한식음식점', '중식음식점', '일식음식점', '양식음식점', '제과점', '패스트푸드점', '치킨전문점',
       '분식전문점', '호프-간이주점', '커피-음료', '일반의원', '치과의원', '당구장', '스포츠클럽', '미용실',
       '네일숍', '피부관리실', '세탁소', '노래방', '슈퍼마켓', '편의점', '반찬가게', '일반의류', '신발',
       '가방', '안경', '시계및귀금속', '의약품', '서적', '화장품', '운동/경기용품', '섬유제품', '화초',
       '가구', '조명용품', '전자상거래업', '외국어학원', '예술학원', '스포츠 강습', '한의원', '골프연습장',
       '가전제품수리', '부동산중개업', '여관', '핸드폰', '미곡판매', '육류판매', '수산물판매', '청과상',
       '의료기기', '문구', '애완동물', '가전제품', '철물점', '인테리어', '일반교습학원', 'PC방',
       '컴퓨터및주변장치판매', '자동차수리', '완구', '자전거 및 기타운송장비', '자동차미용'], dtype=object)

In [13]:
seoul_sales_copy=seoul_sales.copy()

In [19]:
# 요식업
seoul_restaurant_sales = seoul_sales_copy[seoul_sales_copy["서비스_업종_코드"].str.startswith("CS1")]
seoul_restaurant_sales["서비스_업종_코드_명"].unique()

array(['한식음식점', '중식음식점', '일식음식점', '양식음식점', '제과점', '패스트푸드점', '치킨전문점',
       '분식전문점', '호프-간이주점', '커피-음료'], dtype=object)

In [31]:
seoul_restaurant_sales.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,...,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,20241,U,관광특구,3001491,이태원 관광특구,CS100001,한식음식점,1.185751e+10,254192,6.640829e+09,...,69016,38472,129419,109336,1277,73116,86124,33511,28074,16652
1,20241,U,관광특구,3001491,이태원 관광특구,CS100002,중식음식점,1.911647e+09,46140,1.259317e+09,...,18085,4918,21479,21646,151,14443,15206,5918,4892,2514
2,20241,U,관광특구,3001491,이태원 관광특구,CS100003,일식음식점,7.273213e+08,10811,4.611840e+08,...,3596,2734,4855,4682,52,2911,3636,1345,1222,371
3,20241,U,관광특구,3001491,이태원 관광특구,CS100004,양식음식점,1.673468e+10,414722,8.256458e+09,...,135195,83964,228805,173511,3268,151680,156120,45901,31260,14074
4,20241,U,관광특구,3001491,이태원 관광특구,CS100005,제과점,1.255782e+09,96224,7.704259e+08,...,28562,8063,38105,54368,660,29740,28886,12736,12976,7472


In [15]:
# 기타 서비스업(세탁소, 노래방, 의원, 학원 등 기타 개인 서비스 업종)
seoul_etc_sales = seoul_sales_copy[seoul_sales_copy["서비스_업종_코드"].str.startswith("CS2")]
seoul_etc_sales["서비스_업종_코드_명"].unique()

array(['일반의원', '치과의원', '당구장', '스포츠클럽', '미용실', '네일숍', '피부관리실', '세탁소',
       '노래방', '외국어학원', '예술학원', '스포츠 강습', '한의원', '골프연습장', '가전제품수리',
       '부동산중개업', '여관', '일반교습학원', 'PC방', '자동차수리', '자동차미용'], dtype=object)

In [18]:
# 소매업
seoul_retail_sales = seoul_sales_copy[seoul_sales_copy["서비스_업종_코드"].str.startswith("CS3")]  
seoul_retail_sales["서비스_업종_코드_명"].unique()

array(['슈퍼마켓', '편의점', '반찬가게', '일반의류', '신발', '가방', '안경', '시계및귀금속', '의약품',
       '서적', '화장품', '운동/경기용품', '섬유제품', '화초', '가구', '조명용품', '전자상거래업',
       '핸드폰', '미곡판매', '육류판매', '수산물판매', '청과상', '의료기기', '문구', '애완동물',
       '가전제품', '철물점', '인테리어', '컴퓨터및주변장치판매', '완구', '자전거 및 기타운송장비'],
      dtype=object)

In [20]:
seoul_retail_sales.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,...,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
19,20241,U,관광특구,3001491,이태원 관광특구,CS300001,슈퍼마켓,3.187112e+09,255085,2.087682e+09,...,75570,46776,172013,75239,800,63569,87238,32319,33905,29420
20,20241,U,관광특구,3001491,이태원 관광특구,CS300002,편의점,4.912405e+09,790464,3.050870e+09,...,160812,130152,495691,276968,5860,260506,293801,103382,67909,41198
21,20241,U,관광특구,3001491,이태원 관광특구,CS300010,반찬가게,1.860000e+08,5598,1.087791e+08,...,2088,907,3471,1950,11,1209,2066,1106,700,328
22,20241,U,관광특구,3001491,이태원 관광특구,CS300011,일반의류,1.602725e+10,110871,1.012677e+10,...,33011,350,30079,77941,2975,32062,36995,16686,12132,7169
23,20241,U,관광특구,3001491,이태원 관광특구,CS300014,신발,4.362913e+09,26916,2.592180e+09,...,9343,490,15064,10860,169,4692,7483,8806,3309,1466


### 서울시 상권 유동인구  

In [21]:
seoul_street_people=pd.read_csv("../data/서울시 상권분석서비스(길단위인구-상권).csv", encoding="cp949")
seoul_street_people.shape

(6595, 27)

In [22]:
seoul_street_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6595 entries, 0 to 6594
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   기준_년분기_코드         6595 non-null   int64 
 1   상권_구분_코드          6595 non-null   object
 2   상권_구분_코드_명        6595 non-null   object
 3   상권_코드             6595 non-null   int64 
 4   상권_코드_명           6595 non-null   object
 5   총_유동인구_수          6595 non-null   int64 
 6   남성_유동인구_수         6595 non-null   int64 
 7   여성_유동인구_수         6595 non-null   int64 
 8   연령대_10_유동인구_수     6595 non-null   int64 
 9   연령대_20_유동인구_수     6595 non-null   int64 
 10  연령대_30_유동인구_수     6595 non-null   int64 
 11  연령대_40_유동인구_수     6595 non-null   int64 
 12  연령대_50_유동인구_수     6595 non-null   int64 
 13  연령대_60_이상_유동인구_수  6595 non-null   int64 
 14  시간대_00_06_유동인구_수  6595 non-null   int64 
 15  시간대_06_11_유동인구_수  6595 non-null   int64 
 16  시간대_11_14_유동인구_수  6595 non-null   int64 
 17  시간대_14_17_유동인구

In [26]:
seoul_street_people.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,...,시간대_14_17_유동인구_수,시간대_17_21_유동인구_수,시간대_21_24_유동인구_수,월요일_유동인구_수,화요일_유동인구_수,수요일_유동인구_수,목요일_유동인구_수,금요일_유동인구_수,토요일_유동인구_수,일요일_유동인구_수
0,20241,U,관광특구,3001491,이태원 관광특구,2155176,1100997,1054179,140314,532219,...,320185,435781,276720,284230,272781,286789,286045,318414,368486,338431
1,20241,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,6889856,3343470,3546388,335824,1194547,...,1706632,1327800,343472,1040403,1133875,1202348,1130314,1103850,706014,573054
2,20241,U,관광특구,3001493,동대문패션타운 관광특구,3384560,1595828,1788732,188119,645950,...,524693,620807,401942,504854,524755,553766,540100,497858,381895,381333
3,20241,U,관광특구,3001494,종로?청계 관광특구,8440796,4496772,3944023,358062,1523184,...,1808704,1691285,660774,1258576,1328555,1410371,1345939,1334099,998247,765010
4,20241,U,관광특구,3001495,잠실 관광특구,4110178,1976808,2133370,434196,915211,...,653233,879474,477929,570999,570194,589977,580551,598178,621983,578294


### 서울시 상권 직장인구

In [27]:
seoul_working_people=pd.read_csv("../data/서울시 상권분석서비스(직장인구-상권).csv", encoding="cp949")
seoul_working_people.shape

(6549, 26)

In [28]:
seoul_working_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6549 entries, 0 to 6548
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   기준_년분기_코드            6549 non-null   int64 
 1   상권_구분_코드             6549 non-null   object
 2   상권_구분_코드_명           6549 non-null   object
 3   상권_코드                6549 non-null   int64 
 4   상권_코드_명              6549 non-null   object
 5   총_직장_인구_수            6549 non-null   int64 
 6   남성_직장_인구_수           6549 non-null   int64 
 7   여성_직장_인구_수           6549 non-null   int64 
 8   연령대_10_직장_인구_수       6549 non-null   int64 
 9   연령대_20_직장_인구_수       6549 non-null   int64 
 10  연령대_30_직장_인구_수       6549 non-null   int64 
 11  연령대_40_직장_인구_수       6549 non-null   int64 
 12  연령대_50_직장_인구_수       6549 non-null   int64 
 13  연령대_60_이상_직장_인구_수    6549 non-null   int64 
 14  남성연령대_10_직장_인구_수     6549 non-null   int64 
 15  남성연령대_20_직장_인구_수     6549 non-null   int64 
 16  남성연령대_

In [29]:
seoul_working_people.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,...,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,20241,U,관광특구,3001491,이태원 관광특구,28395,15935,12460,8,6830,...,4986,4313,3583,842,0,4627,4869,2052,722,190
1,20241,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,214604,123445,91159,396,43270,...,38123,36717,25448,6374,309,26574,33802,18817,8660,2997
2,20241,U,관광특구,3001493,동대문패션타운 관광특구,33474,16833,16641,273,9661,...,4972,3883,2050,1183,164,5025,3641,2930,3328,1553
3,20241,U,관광특구,3001494,종로?청계 관광특구,47296,26388,20908,670,9566,...,7326,7672,5302,1732,428,5452,6816,5204,2256,752
4,20241,U,관광특구,3001495,잠실 관광특구,111557,65817,45740,195,18343,...,18331,18070,13050,7197,115,9254,10943,10338,11338,3752


### 데이터 병합
> 서울시 추정매출: 서비스 업종 별로 나누기 -> 상권별로 서비스 업종의 매출관련 정보  
> 서울시 유동인구 및 직장인구: 상권권별로 인구관련 정보  

In [30]:
seoul_restaurant_sales["서비스_업종_코드_명"].unique()

array(['한식음식점', '중식음식점', '일식음식점', '양식음식점', '제과점', '패스트푸드점', '치킨전문점',
       '분식전문점', '호프-간이주점', '커피-음료'], dtype=object)

In [ ]:
seoul_restaurant_list = seoul_restaurant_sales["서비스_업종_코드_명"].unique()

results_with_working_population = {}

for restaurant in seoul_restaurant_list:
    temp_df = seoul_restaurant_sales[seoul_restaurant_sales["서비스_업종_코드_명"] == restaurant]
    
    # 평균매출
    mean_sales = temp_df.groupby(["상권_코드_명", "기준_년분기_코드"])[["당월_매출_금액", "월요일_매출_금액", "화요일_매출_금액", "수요일_매출_금액",
                                                            "목요일_매출_금액", "금요일_매출_금액", "토요일_매출_금액", "일요일_매출_금액"]].mean().reset_index()
    mean_sales.rename(columns={"당월_매출_금액": "평균매출"}, inplace=True)
    
    # 유동인구 데이터와 병합
    merged = pd.merge(mean_sales, seoul_street_people, on=["상권_코드_명", "기준_년분기_코드"], how="left")
    # 직장인구 데이터와 병합
    merged_with_working = pd.merge(merged, seoul_working_people, on=["상권_코드_명", "기준_년분기_코드"], how="left")
    
    # 음식점별로 묶기 (한식음식점 데이터프레임, 중식음식점 데이터프레임, ... )
    results_with_working_population[restaurant] = merged_with_working

In [ ]:
# 음식점 이름을 인덱스로 설정
for restaurant, df in results_with_working_population.items():
    df["음식점"] = restaurant

seoul_restaurants = pd.concat(results_with_working_population.values(), ignore_index=True)
seoul_restaurants

In [ ]:
# 파일 불러오기 (cp949 인코딩 사용)
sales_df = pd.read_csv("서울시 상권분석서비스(추정매출-상권).csv", encoding="cp949")
work_df = pd.read_csv("서울시 상권분석서비스(직장인구-상권).csv", encoding="cp949")
street_df = pd.read_csv("서울시 상권분석서비스(길단위인구-상권).csv", encoding="cp949")

# CS1~3 대분류 사전 정의
category_mapping = {
    '요식업': ['한식음식점', '중식음식점', '일식음식점', '양식음식점', '제과점',
             '패스트푸드점', '치킨전문점', '분식전문점', '호프-간이주점', '커피-음료']
}
all_services = sales_df['서비스_업종_코드_명'].unique().tolist()
category_mapping['기타서비스업'] = [s for s in all_services if s not in category_mapping['요식업'] and '서비스' in s]
category_mapping['소매업'] = [s for s in all_services if s not in category_mapping['요식업'] and s not in category_mapping['기타서비스업']]

# 대분류 컬럼 추가
def map_category(service):
    for key, values in category_mapping.items():
        if service in values:
            return key
    return '기타'

sales_df['업종_대분류'] = sales_df['서비스_업종_코드_명'].apply(map_category)

# 대분류별 데이터 분리
seoul_restaurant_sales = sales_df[sales_df['업종_대분류'] == '요식업'].copy()
seoul_etc_sales = sales_df[sales_df['업종_대분류'] == '기타서비스업'].copy()
seoul_retail_sales = sales_df[sales_df['업종_대분류'] == '소매업'].copy()

# 병합 함수 정의
def process_sales_data(df, category_name):
    df['음식점대분류'] = category_name
    mean_sales = df.groupby(['상권_코드_명', '기준_년분기_코드'])[
        ['당월_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액',
         '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    ].mean().reset_index()
    mean_sales.rename(columns={'당월_매출_금액': '평균매출'}, inplace=True)
    merged = pd.merge(mean_sales, street_df, on=['상권_코드_명', '기준_년분기_코드'], how='left')
    merged = pd.merge(merged, work_df, on=['상권_코드_명', '기준_년분기_코드'], how='left')
    merged['음식점대분류'] = category_name
    return merged

# 각 대분류별 병합
restaurant_merged = process_sales_data(seoul_restaurant_sales, '요식업')
etc_merged = process_sales_data(seoul_etc_sales, '기타서비스업')
retail_merged = process_sales_data(seoul_retail_sales, '소매업')

# 통합
final_df = pd.concat([restaurant_merged, etc_merged, retail_merged], ignore_index=True)

# 상권구분코드명 붙이기
area_code_map = sales_df[['상권_코드_명', '상권_구분_코드_명']].drop_duplicates()
final_df = pd.merge(final_df, area_code_map, on='상권_코드_명', how='left')

# 컬럼 순서 정리
base_cols = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '음식점대분류', '평균매출']
weekday_cols = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액',
                '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
population_cols = [col for col in final_df.columns if col not in base_cols + weekday_cols and '매출' not in col and '코드' not in col]
final_df = final_df[base_cols + weekday_cols + population_cols]

final_df

In [ ]:
# 각 대분류별 병합
restaurant_merged = process_sales_data(seoul_restaurant_sales, '요식업')
etc_merged = process_sales_data(seoul_etc_sales, '기타서비스업')
retail_merged = process_sales_data(seoul_retail_sales, '소매업')

# 통합
final_df = pd.concat([restaurant_merged, etc_merged, retail_merged], ignore_index=True)

# 상권구분코드명 붙이기
area_code_map = sales_df[['상권_코드_명', '상권_구분_코드_명']].drop_duplicates()
final_df = pd.merge(final_df, area_code_map, on='상권_코드_명', how='left')

# 컬럼 순서 정리
base_cols = ['기준_년분기_코드', '상권_구분_코드_명', '상권_코드_명', '음식점대분류', '평균매출']
weekday_cols = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액',
                '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
population_cols = [col for col in final_df.columns if col not in base_cols + weekday_cols and '매출' not in col and '코드' not in col]
final_df = final_df[base_cols + weekday_cols + population_cols]

final_df

# CSV 파일로 저장
#final_df.to_csv("최종_상권_업종_인구_통합.csv", index=False)